In [ ]:
def regression_model_setup():
  class EfficientNet9ChannelsModel(nn.Module):
        def __init__(self, pretrained=True):
            super(EfficientNet9ChannelsModel, self).__init__()
            # Load the pre-trained EfficientNet model
            self.efficientnet = models.efficientnet_b0(pretrained=pretrained)

            # Modify the first convolution layer to accept 3 input channels
            self.efficientnet.features[0][0] = nn.Conv2d(
                3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
            )

            # for param in self.efficientnet.parameters():
            #     param.requires_grad = False

            self.dropout = nn.Dropout(p=0.5)

            # Modify the classifier to output 1 value (for regression)
            self.efficientnet.classifier = nn.Sequential(
                nn.Linear(self.efficientnet.classifier[1].in_features, 1)
            )

            # for param in self.efficientnet.classifier.parameters():
            #     param.requires_grad = True

        def forward(self, x):
            x = self.efficientnet.features(x)
            x = self.efficientnet.avgpool(x)  # Use the pre-defined avgpool
            x = torch.flatten(x, 1)
            x = self.dropout(x)
            x = self.efficientnet.classifier(x)
            return x

  model = EfficientNet9ChannelsModel(pretrained=True) #MobileNet9ChannelsModel(pretrained=True)
  criterion = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

  return model, criterion, optimizer#, scheduler

In [ ]:
def load_model(save_path="regression_model.pth"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model, _, _ = regression_model_setup()  # Initialize the model structure
    model.load_state_dict(torch.load(save_path, map_location=device))  # Load the state dictionary
    model.to(device)
    model.eval()  # Set the model to evaluation mode
    print(f"Model loaded from {save_path}")
    return model

In [ ]:
def created_model_input(results):
  nail_class = 0
  blue_circle_class_id = 1

  top_nail_bounds = {}
  normalized = {}
  normalized_padded = {}

  transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Resize((224, 224))
    ])

  for result in results:
    image_name = result.path.split('/')[-1]
    boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes (x_min, y_min, x_max, y_max)
    scores = result.boxes.conf.cpu().numpy()  # Confidence scores
    classes = result.boxes.cls.cpu().numpy()  # Class IDs

    nail_boxes = [(box, score) for box, score, cls in zip(boxes, scores, classes) if cls == nail_class]
    nail_boxes = sorted(nail_boxes, key=lambda x: x[1], reverse=True)[:3]
    top_nail_bounds[image_name] = nail_boxes

    normalized_images = []
    image_path = result.path
    image = cv2.imread(image_path)

    blue_circle_box = next(
          (box for box, cls in zip(result.boxes.xyxy.cpu().numpy(), result.boxes.cls.cpu().numpy())
          if cls == blue_circle_class_id),
          None
          )
    # if blue_circle_box is None:
    #   print(image_name)
    #   continue

      # Compute the center pigment value
    x_min, y_min, x_max, y_max = map(int, blue_circle_box)
    center_x = (x_min + x_max) // 2
    center_y = (y_min + y_max) // 2
    center_pixel_value = image[center_y, center_x]  # BGR pixel value

    for (box, score) in top_nail_bounds.get(image_name, []):
      x_min, y_min, x_max, y_max = map(int, box)
      nail_region = image[y_min:y_max, x_min:x_max]

        # Normalize pixel values by center pigment
      normalized_nail = nail_region / center_pixel_value
      normalized_images.append(nail_region)#(normalized_nail)

    normalized[image_name] = normalized_images

    for key, images in normalized.items():
      while len(images) < 3:
          images.append(images[-1])  # Duplicate the last image until there are 3
      normalized_padded[key] = images[:3]

    combined_image = torch.cat([transform(img) for img in images], dim=2)

  return combined_image

In [ ]:
def preprocess(image_path, seg_model_path):
  model = YOLO(seg_model_path)
  result = model(image_path, save=True)
  combined_img = created_model_input(result)
  return combined_img

In [ ]:
!pip install ultralytics

from google.colab import drive
from ultralytics import YOLO
import pandas as pd
import torch
from torchvision import transforms
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import os
import shutil
from sklearn.model_selection import train_test_split
import zipfile
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset
import random
import numpy as np
import torch
from torchvision import transforms

drive.mount('/content/drive')
image_path = "/content/drive/MyDrive/AI_ML_Project/Right_Fingernail/test_1/1709970910465.jpg"  # Replace with the path to your new image
seg_model_path = '/content/drive/MyDrive/AI_ML_Project/Right_Fingernail/best.pt'
input_tensor = preprocess(image_path, seg_model_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

image 1/1 /content/drive/MyDrive/AI_ML_Project/Right_Fingernail/test_1/1709970910465.jpg: 1024x1024 4 nails, 1 colorcard, 1 bluecircle, 566.4ms
Speed: 12.8ms preprocess, 566.4ms inference, 1.2ms postprocess per image at shape (1, 3, 1024, 1024)
Results saved to runs/detect/predict14


In [ ]:
reg_model = load_model('/content/drive/MyDrive/AI_ML_Project/BEST/regression_model.pth')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_tensor = input_tensor.unsqueeze(0)
input_tensor = input_tensor.to(device)
with torch.no_grad():
    prediction = reg_model(input_tensor)

print(f"Predicted Hemoglobin (Hb) value: {prediction.item():.4f}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model loaded from /content/drive/MyDrive/AI_ML_Project/BEST/regression_model.pth
Predicted Hemoglobin (Hb) value: 11.9568
